In [ ]:
playlist_urls = []
for i in [0]:
    playlist_urls.append(\
        f'https://sfs-msc-pub-lq-05.navigator.dot.ga.gov/rtplive/GDOT-CCTV-0'
        f'{500 + i}/playlist.m3u8')
playlist_urls

In [ ]:
import urllib.request
import requests
import re
import urllib
from pathlib import Path
import subprocess
import time
import os

wait = 10
while True:
    start = time.time()
    for url in playlist_urls:
        num = base_url[-3:-1]
        Path(f'rawmedia/{num}/').mkdir(exist_ok=True, parents=True)
        Path(f'media/{num}/').mkdir(exist_ok=True, parents=True)
        num_mp4 = len(os.listdir(f'media/{num}'))
        print(num_mp4)
        base_url = url[:url.rfind('/') + 1]
        playlist = requests.get(url).text
        file_name = re.findall(r'chunklist_w.*?\.m3u8', playlist)
        if (len(file_name) == 0):
            continue
        file_name = file_name[-1]
        chunklist_url = base_url + file_name
        chunklist = requests.get(chunklist_url).text
        media_files = re.findall(r'media_w.*?\.ts', chunklist)
        media_concat = 'concat:'
        for media_file in media_files:
            urllib.request.urlretrieve(base_url + media_file, f'rawmedia/{num}/{media_file}')
            subprocess.run(['ffmpeg', '-i', 
                            f'rawmedia/{num}/{media_file}', 
                            f'media/{num}/media{num_mp4}.mp4'])
            num_mp4 += 1
        print(f'time elapsed {start - time.time()}')
        time.sleep(start - time.time() + wait)
